# Multi-Agent HR System - Complete Labs

## Overview
This notebook demonstrates various multi-agent patterns for HR use cases using LangChain 1.0 and OpenAI:

- **Lab 1**: Basic Multi-Agent (No Tools, No Memory) - Resume Screening
- **Lab 2**: Multi-Agent with Tools (No Memory) - Candidate Processing  
- **Lab 3**: Multi-Agent with Tools and Memory - Interview Coordination
- **Lab 4**: Human-in-the-Loop - Offer Approval Workflow
- **Lab 5**: LangGraph Subgraphs - Complete Hiring Pipeline

### HR Use Case
**Scenario**: TechCorp India - Automated Hiring System
- Multiple specialized agents handle different hiring stages
- Integration with HR tools (database, calendar, email)
- Collaborative decision-making with human oversight

## Setup and Installation

In [ ]:
# Install required packages
!pip install -q langchain langchain-openai langgraph langchain-core python-dotenv

In [ ]:
# Import libraries
import os
from dotenv import load_dotenv
from typing import TypedDict, List, Annotated, Literal
import operator
from datetime import datetime, timedelta
import json

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.tools import tool

# Load environment variables
load_dotenv()

# Verify OpenAI API key
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("Please set OPENAI_API_KEY in your environment")

print("✓ Setup complete!")

## Sample HR Data - TechCorp India

Realistic Indian HR dataset for our labs

In [ ]:
# Sample candidate resume
SAMPLE_RESUME = """
PRIYA SHARMA
Senior Software Engineer
priya.sharma@email.com | +91-98765-43210 | Bengaluru, Karnataka
LinkedIn: linkedin.com/in/priyasharma | GitHub: github.com/priyasharma

SUMMARY
Results-driven software engineer with 6 years of experience in full-stack development,
specializing in Python, React, and cloud-native architectures. Proven track record of
building scalable microservices and leading agile teams. Passionate about mentoring
and delivering high-quality software solutions.

EXPERIENCE

Senior Software Engineer | InfoTech Solutions Pvt Ltd, Bengaluru | Jan 2021 - Present
- Led a team of 4 developers in building a microservices-based e-commerce platform
- Reduced API latency by 45% through optimization and caching strategies
- Implemented CI/CD pipeline using Jenkins and Docker, reducing deployment time by 50%
- Mentored 3 junior developers and conducted technical interviews
- Tech Stack: Python, FastAPI, React, PostgreSQL, Redis, AWS, Docker, Kubernetes

Software Engineer | Digital Innovations India, Pune | Jun 2018 - Dec 2020
- Developed customer-facing web applications serving 500K+ monthly active users
- Built RESTful APIs and integrated third-party payment gateways (Razorpay, PayU)
- Collaborated with cross-functional teams in Agile/Scrum environment
- Tech Stack: Python, Django, JavaScript, MySQL, AWS EC2, S3

Junior Developer | CodeCraft Technologies, Hyderabad | Jul 2016 - May 2018
- Maintained and enhanced legacy applications
- Developed internal tools to automate manual processes
- Participated in code reviews and testing activities

EDUCATION
B.Tech in Computer Science and Engineering | BITS Pilani | 2016
CGPA: 8.5/10

SKILLS
Languages: Python, JavaScript, TypeScript, SQL, Java
Frameworks: FastAPI, Django, React, Next.js, Flask
Cloud & DevOps: AWS (EC2, S3, Lambda, RDS), Docker, Kubernetes, Jenkins
Databases: PostgreSQL, MySQL, MongoDB, Redis
Tools: Git, JIRA, Confluence, Postman

CERTIFICATIONS
- AWS Certified Solutions Architect - Associate (2022)
- Python for Data Science - NPTEL (2020)
"""

JOB_DESCRIPTION = """
Senior Backend Engineer
TechCorp India Pvt Ltd | Bengaluru

About the Role:
We are seeking an experienced Senior Backend Engineer to join our growing engineering team.
You will be responsible for designing and building scalable backend systems that power our
fintech applications serving millions of users across India.

Key Responsibilities:
- Design and develop high-performance backend services using Python/FastAPI
- Build and maintain microservices architecture on AWS/GCP
- Lead technical discussions and mentor junior engineers
- Ensure code quality, testing, and documentation standards
- Collaborate with product and frontend teams

Required Qualifications:
- 5+ years of backend development experience
- Strong proficiency in Python and related frameworks (FastAPI/Django)
- Experience with cloud platforms (AWS/GCP)
- Solid understanding of databases (SQL and NoSQL)
- Experience with containerization (Docker/Kubernetes)
- Knowledge of CI/CD pipelines

Preferred Qualifications:
- Experience in fintech or payment systems
- AWS/GCP certifications
- Contribution to open-source projects
- Experience leading small teams

What We Offer:
- Competitive salary (₹25-35 LPA)
- Stock options
- Health insurance for family
- Flexible work arrangements
- Learning and development budget
"""

# Mock HR Database
HR_DATABASE = {
    "employees": [
        {"id": "EMP001", "name": "Rahul Verma", "role": "Engineering Manager", "email": "rahul.verma@techcorp.in"},
        {"id": "EMP002", "name": "Anjali Patel", "role": "HR Manager", "email": "anjali.patel@techcorp.in"},
        {"id": "EMP003", "name": "Vikram Singh", "role": "Tech Lead", "email": "vikram.singh@techcorp.in"},
        {"id": "EMP004", "name": "Sneha Reddy", "role": "Senior Engineer", "email": "sneha.reddy@techcorp.in"},
    ],
    "candidates": [
        {
            "id": "CAN001",
            "name": "Priya Sharma",
            "email": "priya.sharma@email.com",
            "phone": "+91-98765-43210",
            "position": "Senior Backend Engineer",
            "status": "screening",
            "resume_url": "https://storage.techcorp.in/resumes/priya_sharma.pdf"
        },
        {
            "id": "CAN002",
            "name": "Arjun Mehta",
            "email": "arjun.mehta@email.com",
            "phone": "+91-98123-45678",
            "position": "Senior Backend Engineer",
            "status": "interview_scheduled",
            "resume_url": "https://storage.techcorp.in/resumes/arjun_mehta.pdf"
        }
    ],
    "interview_slots": [
        {"date": "2025-10-15", "time": "10:00 AM", "interviewer": "Rahul Verma", "available": True},
        {"date": "2025-10-15", "time": "2:00 PM", "interviewer": "Vikram Singh", "available": True},
        {"date": "2025-10-16", "time": "11:00 AM", "interviewer": "Sneha Reddy", "available": True},
    ]
}

print("HR Database initialized with:")
print(f"- {len(HR_DATABASE['employees'])} employees")
print(f"- {len(HR_DATABASE['candidates'])} candidates")
print(f"- {len(HR_DATABASE['interview_slots'])} interview slots")

---

# LAB 1: Basic Multi-Agent (No Tools, No Memory)

**Pattern**: Simple agent coordination using LangChain 1.0

**Use Case**: Resume screening with specialized agents:
- **Resume Reviewer**: Analyzes experience and qualifications
- **Skills Assessor**: Evaluates technical skills match
- **Culture Fit Analyst**: Assesses company culture alignment
- **Hiring Coordinator**: Synthesizes recommendations

In [ ]:
# Initialize LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Define state for agent collaboration
class ScreeningState(TypedDict):
    candidate_name: str
    position: str
    resume: str
    job_description: str
    resume_analysis: str
    skills_assessment: str
    culture_fit: str
    final_decision: str
    messages: List[str]

# Agent 1: Resume Reviewer
def resume_reviewer_agent(state: ScreeningState) -> ScreeningState:
    """
    Analyzes resume for relevant experience and qualifications
    """
    prompt = f"""
You are an expert Resume Reviewer for TechCorp India.

Position: {state['position']}
Candidate: {state['candidate_name']}

Resume:
{state['resume']}

Job Requirements:
{state['job_description']}

Provide a structured analysis:
1. Years of relevant experience
2. Educational background match
3. Key qualifications aligning with role
4. Notable achievements
5. Any concerns or gaps

Keep your analysis concise and professional.
"""
    
    messages = [HumanMessage(content=prompt)]
    response = llm.invoke(messages)
    
    state['resume_analysis'] = response.content
    state['messages'].append(f"[Resume Reviewer] Analysis complete")
    
    return state

# Agent 2: Skills Assessor
def skills_assessor_agent(state: ScreeningState) -> ScreeningState:
    """
    Evaluates technical skills against job requirements
    """
    prompt = f"""
You are a Technical Skills Assessor for TechCorp India.

Position: {state['position']}
Candidate: {state['candidate_name']}

Resume:
{state['resume']}

Job Requirements:
{state['job_description']}

Previous Analysis:
{state['resume_analysis']}

Evaluate the candidate's technical skills:
1. Programming languages proficiency match
2. Framework and technology stack alignment
3. Cloud platform experience (AWS/GCP)
4. DevOps and tooling knowledge
5. Skills gap analysis
6. Overall technical fit score (1-10)

Be specific and objective.
"""
    
    messages = [HumanMessage(content=prompt)]
    response = llm.invoke(messages)
    
    state['skills_assessment'] = response.content
    state['messages'].append(f"[Skills Assessor] Assessment complete")
    
    return state

# Agent 3: Culture Fit Analyst
def culture_fit_analyst(state: ScreeningState) -> ScreeningState:
    """
    Assesses candidate's alignment with company culture
    """
    prompt = f"""
You are a Culture Fit Analyst for TechCorp India.

Position: {state['position']}
Candidate: {state['candidate_name']}

Resume:
{state['resume']}

Previous Analyses:
Resume Review: {state['resume_analysis']}
Skills Assessment: {state['skills_assessment']}

Company Culture: Collaborative, innovation-driven, mentorship-focused, 
fast-paced startup environment, values continuous learning.

Assess culture fit based on:
1. Leadership and mentoring experience
2. Collaboration indicators
3. Learning and growth mindset
4. Startup/fast-paced environment experience
5. Innovation and contribution indicators
6. Culture fit score (1-10)

Provide insights, not just scores.
"""
    
    messages = [HumanMessage(content=prompt)]
    response = llm.invoke(messages)
    
    state['culture_fit'] = response.content
    state['messages'].append(f"[Culture Fit Analyst] Analysis complete")
    
    return state

# Agent 4: Hiring Coordinator (Synthesizer)
def hiring_coordinator(state: ScreeningState) -> ScreeningState:
    """
    Synthesizes all analyses and makes final recommendation
    """
    prompt = f"""
You are the Hiring Coordinator for TechCorp India.

Position: {state['position']}
Candidate: {state['candidate_name']}

You have received the following analyses:

RESUME ANALYSIS:
{state['resume_analysis']}

SKILLS ASSESSMENT:
{state['skills_assessment']}

CULTURE FIT ANALYSIS:
{state['culture_fit']}

Synthesize the above analyses and provide:
1. Overall summary
2. Key strengths
3. Areas of concern (if any)
4. Final recommendation (STRONG PROCEED / PROCEED / MAYBE / REJECT)
5. Next steps if proceeding

Be decisive and clear in your recommendation.
"""
    
    messages = [HumanMessage(content=prompt)]
    response = llm.invoke(messages)
    
    state['final_decision'] = response.content
    state['messages'].append(f"[Hiring Coordinator] Final decision ready")
    
    return state

# Orchestrate the multi-agent workflow
def run_screening_workflow(candidate_name: str, resume: str, job_desc: str, position: str):
    """
    Run the complete screening workflow
    """
    print("\n" + "="*80)
    print(f"Starting Resume Screening for: {candidate_name}")
    print("="*80 + "\n")
    
    # Initialize state
    state: ScreeningState = {
        "candidate_name": candidate_name,
        "position": position,
        "resume": resume,
        "job_description": job_desc,
        "resume_analysis": "",
        "skills_assessment": "",
        "culture_fit": "",
        "final_decision": "",
        "messages": []
    }
    
    # Execute agents in sequence
    state = resume_reviewer_agent(state)
    print(state['messages'][-1])
    
    state = skills_assessor_agent(state)
    print(state['messages'][-1])
    
    state = culture_fit_analyst(state)
    print(state['messages'][-1])
    
    state = hiring_coordinator(state)
    print(state['messages'][-1])
    
    return state

# Test Lab 1
print("\n🧪 Lab 1: Basic Multi-Agent (No Tools, No Memory)\n")
result = run_screening_workflow(
    candidate_name="Priya Sharma",
    resume=SAMPLE_RESUME,
    job_desc=JOB_DESCRIPTION,
    position="Senior Backend Engineer"
)

print("\n" + "="*80)
print("FINAL HIRING DECISION")
print("="*80)
print(result['final_decision'])

---

# LAB 2: Multi-Agent with Tools (No Memory)

**Pattern**: Tool Calling - agents use tools to access HR systems

**Tools**:
- `search_hr_database`: Query employee and candidate information
- `check_interview_slots`: Check interviewer availability
- `send_email`: Send email notifications
- `update_candidate_status`: Update candidate status in ATS

In [ ]:
# Define HR Tools

@tool
def search_hr_database(query_type: str, search_term: str) -> str:
    """
    Search the HR database for employee or candidate information.
    
    Args:
        query_type: Type of search - 'employee' or 'candidate'
        search_term: Name, ID, or role to search for
    
    Returns:
        JSON string of matching results
    """
    results = []
    
    if query_type == "employee":
        for emp in HR_DATABASE["employees"]:
            if search_term.lower() in emp["name"].lower() or search_term.lower() in emp["role"].lower():
                results.append(emp)
    
    elif query_type == "candidate":
        for cand in HR_DATABASE["candidates"]:
            if search_term.lower() in cand["name"].lower() or search_term.lower() in cand["position"].lower():
                results.append(cand)
    
    return json.dumps(results, indent=2)

@tool
def check_interview_slots(interviewer_name: str = None, date: str = None) -> str:
    """
    Check available interview slots.
    
    Args:
        interviewer_name: Optional - filter by interviewer name
        date: Optional - filter by date (YYYY-MM-DD)
    
    Returns:
        JSON string of available slots
    """
    slots = HR_DATABASE["interview_slots"]
    
    if interviewer_name:
        slots = [s for s in slots if interviewer_name.lower() in s["interviewer"].lower()]
    
    if date:
        slots = [s for s in slots if s["date"] == date]
    
    available_slots = [s for s in slots if s["available"]]
    
    return json.dumps(available_slots, indent=2)

@tool
def send_email(to_email: str, subject: str, body: str) -> str:
    """
    Send an email notification.
    
    Args:
        to_email: Recipient email address
        subject: Email subject
        body: Email body content
    
    Returns:
        Confirmation message
    """
    # In real scenario, this would send actual email
    return f"Email sent successfully to {to_email}\nSubject: {subject}\n\n[SIMULATED - No actual email sent]"

@tool
def update_candidate_status(candidate_id: str, new_status: str, notes: str = "") -> str:
    """
    Update candidate status in the ATS (Applicant Tracking System).
    
    Args:
        candidate_id: Candidate ID (e.g., CAN001)
        new_status: New status (screening, interview_scheduled, offer, rejected)
        notes: Optional notes about the status change
    
    Returns:
        Confirmation message
    """
    for candidate in HR_DATABASE["candidates"]:
        if candidate["id"] == candidate_id:
            old_status = candidate["status"]
            candidate["status"] = new_status
            return f"Updated {candidate['name']}: {old_status} → {new_status}\nNotes: {notes}"
    
    return f"Candidate {candidate_id} not found"

# List of all tools
hr_tools = [search_hr_database, check_interview_slots, send_email, update_candidate_status]

print("✓ HR Tools defined:")
for tool in hr_tools:
    print(f"  - {tool.name}: {tool.description}")

In [ ]:
# Create Tool-Calling Agents

# Agent 1: Recruitment Coordinator with Tools
recruitment_coordinator_prompt = ChatPromptTemplate.from_messages([
    ("system", """
You are a Recruitment Coordinator at TechCorp India.

Your responsibilities:
- Search for candidate information in HR database
- Check interviewer availability
- Schedule interviews
- Send email notifications to candidates and interviewers
- Update candidate status in the system

Always use the available tools to get accurate, real-time information.
Be professional and efficient in your communication.
"""),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

recruitment_agent = create_tool_calling_agent(llm, hr_tools, recruitment_coordinator_prompt)
recruitment_executor = AgentExecutor(agent=recruitment_agent, tools=hr_tools, verbose=True)

print("✓ Recruitment Coordinator Agent created with HR tools")

In [ ]:
# Test Lab 2: Multi-Agent with Tools

print("\n🧪 Lab 2: Multi-Agent with Tools (No Memory)\n")
print("="*80)

# Task 1: Find candidate and schedule interview
task1 = """
I need to schedule an interview for candidate Priya Sharma (ID: CAN001) for the 
Senior Backend Engineer position.

Please:
1. Look up the candidate details
2. Find available interview slots with Rahul Verma or Vikram Singh
3. Schedule the interview for the earliest available slot
4. Update the candidate status to 'interview_scheduled'
5. Send confirmation emails to both the candidate and interviewer
"""

result1 = recruitment_executor.invoke({"input": task1})
print("\n" + "="*80)
print("Task Result:")
print(result1["output"])

---

# LAB 3: Multi-Agent with Tools and Memory

**Pattern**: Persistent memory with checkpointing using LangGraph

**Features**:
- Conversation memory across interactions
- State persistence using checkpointing
- Context retention for multi-turn conversations

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Define state with memory
class HRAgentState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    candidate_context: dict

# Create agent with memory
memory = MemorySaver()

hr_agent_with_memory = create_react_agent(
    llm,
    tools=hr_tools,
    state_schema=HRAgentState,
    checkpointer=memory
)

print("✓ HR Agent with memory created")

In [ ]:
# Test Lab 3: Multi-turn conversation with memory

print("\n🧪 Lab 3: Multi-Agent with Tools and Memory\n")
print("="*80)

# Configuration for thread-based memory
config = {"configurable": {"thread_id": "hiring_thread_priya_001"}}

# Turn 1: Initial query
print("\n--- Turn 1: Initial Query ---")
response1 = hr_agent_with_memory.invoke(
    {"messages": [HumanMessage(content="Can you find information about candidate Priya Sharma?")]},
    config
)
print(response1["messages"][-1].content)

# Turn 2: Follow-up (agent remembers context)
print("\n--- Turn 2: Follow-up Question (Testing Memory) ---")
response2 = hr_agent_with_memory.invoke(
    {"messages": [HumanMessage(content="What interview slots are available for her next week?")]},
    config
)
print(response2["messages"][-1].content)

# Turn 3: Action based on previous context
print("\n--- Turn 3: Action with Context ---")
response3 = hr_agent_with_memory.invoke(
    {"messages": [HumanMessage(content="Schedule her for the first available slot and send her a confirmation email.")]},
    config
)
print(response3["messages"][-1].content)

# Turn 4: Verify agent remembers everything
print("\n--- Turn 4: Memory Verification ---")
response4 = hr_agent_with_memory.invoke(
    {"messages": [HumanMessage(content="Can you summarize what we've done for this candidate so far?")]},
    config
)
print(response4["messages"][-1].content)

---

# LAB 4: Human-in-the-Loop Workflow

**Pattern**: Approval workflow with human intervention

**Use Case**: Offer approval process
- Agent prepares offer details
- Human reviews and approves/rejects
- Agent proceeds based on approval

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage

# Define state for offer approval workflow
class OfferApprovalState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    candidate_name: str
    position: str
    proposed_salary: str
    offer_details: str
    approval_status: str
    human_feedback: str

# Node 1: Prepare Offer
def prepare_offer(state: OfferApprovalState):
    """
    AI agent prepares offer details
    """
    prompt = f"""
You are an HR Compensation Specialist at TechCorp India.

Prepare a job offer for:
- Candidate: {state.get('candidate_name', 'Priya Sharma')}
- Position: {state.get('position', 'Senior Backend Engineer')}
- Proposed Salary: {state.get('proposed_salary', '₹30 LPA')}

Create a comprehensive offer package including:
1. Base salary
2. Performance bonus structure
3. Stock options (if applicable)
4. Benefits (health insurance, learning budget, etc.)
5. Start date
6. Other perks

Format it professionally for management review.
"""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    
    return {
        "offer_details": response.content,
        "approval_status": "pending",
        "messages": [HumanMessage(content="Offer prepared and ready for approval")]
    }

# Node 2: Human Approval (This is where we pause for human input)
def human_approval(state: OfferApprovalState):
    """
    Pause for human approval
    """
    print("\n" + "="*80)
    print("OFFER PENDING APPROVAL")
    print("="*80)
    print(state["offer_details"])
    print("\n" + "="*80)
    
    # In real implementation, this would trigger an interrupt
    # For demo, we'll simulate human input
    approval = input("\nApprove this offer? (yes/no/modify): ").strip().lower()
    
    if approval == "yes":
        return {
            "approval_status": "approved",
            "human_feedback": "Offer approved",
            "messages": [HumanMessage(content="Offer approved by manager")]
        }
    elif approval == "no":
        feedback = input("Reason for rejection: ")
        return {
            "approval_status": "rejected",
            "human_feedback": feedback,
            "messages": [HumanMessage(content=f"Offer rejected: {feedback}")]
        }
    else:
        feedback = input("What modifications are needed?: ")
        return {
            "approval_status": "needs_modification",
            "human_feedback": feedback,
            "messages": [HumanMessage(content=f"Modifications requested: {feedback}")]
        }

# Node 3: Process Approval
def process_approval(state: OfferApprovalState):
    """
    Take action based on approval status
    """
    if state["approval_status"] == "approved":
        # Send offer letter
        action = f"""Offer approved! Next steps:
1. Generate formal offer letter
2. Send to candidate: {state.get('candidate_name', 'Priya Sharma')}
3. Update ATS status to 'offer_sent'
4. Set follow-up reminder for 3 days
"""
    elif state["approval_status"] == "rejected":
        action = f"""Offer rejected. Reason: {state['human_feedback']}
No further action taken. Candidate remains in pipeline.
"""
    else:
        action = f"""Modifications requested: {state['human_feedback']}
Sending back to compensation team for revision.
"""
    
    return {
        "messages": [HumanMessage(content=action)]
    }

# Build workflow graph
def should_continue(state: OfferApprovalState) -> Literal["process", "end"]:
    if state.get("approval_status") in ["approved", "rejected"]:
        return "process"
    return "end"

# Create graph
workflow = StateGraph(OfferApprovalState)

# Add nodes
workflow.add_node("prepare", prepare_offer)
workflow.add_node("approve", human_approval)
workflow.add_node("process", process_approval)

# Add edges
workflow.add_edge(START, "prepare")
workflow.add_edge("prepare", "approve")
workflow.add_conditional_edges(
    "approve",
    should_continue,
    {"process": "process", "end": END}
)
workflow.add_edge("process", END)

# Compile
hitl_app = workflow.compile()

print("✓ Human-in-the-Loop workflow created")

In [ ]:
# Test Lab 4: Human-in-the-Loop

print("\n🧪 Lab 4: Human-in-the-Loop Workflow\n")
print("="*80)

initial_state = {
    "messages": [],
    "candidate_name": "Priya Sharma",
    "position": "Senior Backend Engineer",
    "proposed_salary": "₹32 LPA",
    "offer_details": "",
    "approval_status": "",
    "human_feedback": ""
}

# Run workflow (will pause for human input)
result = hitl_app.invoke(initial_state)

print("\n" + "="*80)
print("WORKFLOW COMPLETE")
print("="*80)
for msg in result["messages"]:
    print(f"\n{msg.content}")

---

# LAB 5: LangGraph Subgraphs - Complete Hiring Pipeline

**Pattern**: Multi-agent system using subgraphs

**Architecture**:
- **Parent Graph**: Overall hiring orchestration
- **Subgraph 1**: Screening Team (multiple specialized agents)
- **Subgraph 2**: Interview Coordination Team
- **Subgraph 3**: Offer Management Team

This demonstrates the full power of LangGraph with nested agent teams!

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import Literal

# ============================================================================
# SUBGRAPH 1: Screening Team
# ============================================================================

class ScreeningSubgraphState(TypedDict):
    candidate_name: str
    resume: str
    job_description: str
    screening_result: str  # shared key with parent
    technical_score: int
    culture_score: int

def resume_screener(state: ScreeningSubgraphState):
    """Quick resume screening"""
    prompt = f"""Quickly screen this resume for {state['candidate_name']}:
{state['resume'][:500]}...

Job: {state['job_description'][:300]}...

Provide: PASS/FAIL and brief reason (2 sentences max)."""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    return {"screening_result": response.content}

def technical_evaluator(state: ScreeningSubgraphState):
    """Evaluate technical skills"""
    if "FAIL" in state.get("screening_result", ""):
        return {"technical_score": 0}
    
    prompt = f"""Rate technical skills (1-10) for:
{state['resume'][:400]}...

Just give a score and one-line justification."""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    # Extract score (simplified)
    score = 7  # In real scenario, parse from response
    return {"technical_score": score}

def culture_evaluator(state: ScreeningSubgraphState):
    """Evaluate culture fit"""
    if "FAIL" in state.get("screening_result", ""):
        return {"culture_score": 0}
    
    score = 8  # Simplified
    return {"culture_score": score}

def screening_decision(state: ScreeningSubgraphState):
    """Make final screening decision"""
    tech_score = state.get("technical_score", 0)
    culture_score = state.get("culture_score", 0)
    avg_score = (tech_score + culture_score) / 2
    
    if avg_score >= 6:
        result = f"PROCEED TO INTERVIEW (Tech: {tech_score}/10, Culture: {culture_score}/10)"
    else:
        result = f"REJECT (Tech: {tech_score}/10, Culture: {culture_score}/10)"
    
    return {"screening_result": result}

# Build screening subgraph
screening_builder = StateGraph(ScreeningSubgraphState)
screening_builder.add_node("screen", resume_screener)
screening_builder.add_node("tech_eval", technical_evaluator)
screening_builder.add_node("culture_eval", culture_evaluator)
screening_builder.add_node("decision", screening_decision)

screening_builder.add_edge(START, "screen")
screening_builder.add_edge("screen", "tech_eval")
screening_builder.add_edge("tech_eval", "culture_eval")
screening_builder.add_edge("culture_eval", "decision")

screening_subgraph = screening_builder.compile()

print("✓ Screening Subgraph created")

In [ ]:
# ============================================================================
# SUBGRAPH 2: Interview Coordination Team
# ============================================================================

class InterviewSubgraphState(TypedDict):
    candidate_name: str
    interview_scheduled: bool  # shared with parent
    interview_details: str

def find_interviewers(state: InterviewSubgraphState):
    """Find available interviewers"""
    # Simulated
    interviewers = "Rahul Verma, Vikram Singh"
    return {"interview_details": f"Interviewers available: {interviewers}"}

def schedule_interview(state: InterviewSubgraphState):
    """Schedule the interview"""
    details = f"""
Interview scheduled for {state['candidate_name']}
Date: 2025-10-15
Time: 10:00 AM IST
Panel: Rahul Verma (Technical), Vikram Singh (System Design)
Duration: 90 minutes
"""
    return {
        "interview_scheduled": True,
        "interview_details": details
    }

# Build interview subgraph
interview_builder = StateGraph(InterviewSubgraphState)
interview_builder.add_node("find_panel", find_interviewers)
interview_builder.add_node("schedule", schedule_interview)

interview_builder.add_edge(START, "find_panel")
interview_builder.add_edge("find_panel", "schedule")

interview_subgraph = interview_builder.compile()

print("✓ Interview Subgraph created")

In [ ]:
# ============================================================================
# PARENT GRAPH: Overall Hiring Pipeline
# ============================================================================

class HiringPipelineState(TypedDict):
    candidate_name: str
    resume: str
    job_description: str
    screening_result: str  # shared with screening subgraph
    interview_scheduled: bool  # shared with interview subgraph
    interview_details: str
    final_status: str

def start_pipeline(state: HiringPipelineState):
    """Initialize the hiring pipeline"""
    return {
        "final_status": f"Starting hiring pipeline for {state['candidate_name']}"
    }

def should_interview(state: HiringPipelineState) -> Literal["interview", "reject"]:
    """Route based on screening result"""
    if "PROCEED" in state.get("screening_result", ""):
        return "interview"
    return "reject"

def reject_candidate(state: HiringPipelineState):
    """Handle rejection"""
    return {
        "final_status": f"Candidate {state['candidate_name']} rejected at screening stage.\nReason: {state['screening_result']}"
    }

def finalize_pipeline(state: HiringPipelineState):
    """Finalize the pipeline"""
    if state.get("interview_scheduled"):
        status = f"""
✓ Candidate {state['candidate_name']} successfully processed!

Screening: {state['screening_result']}

Interview Details:
{state.get('interview_details', 'Not scheduled')}

Next steps: Interview panel will evaluate and provide feedback.
"""
    else:
        status = state.get("final_status", "Pipeline complete")
    
    return {"final_status": status}

# Build parent graph
parent_builder = StateGraph(HiringPipelineState)

# Add nodes (including subgraphs)
parent_builder.add_node("start", start_pipeline)
parent_builder.add_node("screening_team", screening_subgraph)  # Subgraph as node!
parent_builder.add_node("interview_team", interview_subgraph)  # Subgraph as node!
parent_builder.add_node("reject", reject_candidate)
parent_builder.add_node("finalize", finalize_pipeline)

# Add edges
parent_builder.add_edge(START, "start")
parent_builder.add_edge("start", "screening_team")

# Conditional routing after screening
parent_builder.add_conditional_edges(
    "screening_team",
    should_interview,
    {
        "interview": "interview_team",
        "reject": "reject"
    }
)

parent_builder.add_edge("interview_team", "finalize")
parent_builder.add_edge("reject", "finalize")
parent_builder.add_edge("finalize", END)

# Compile the complete pipeline
hiring_pipeline = parent_builder.compile()

print("✓ Complete Hiring Pipeline with Subgraphs created!")

In [ ]:
# Test Lab 5: Complete Pipeline with Subgraphs

print("\n🧪 Lab 5: LangGraph Subgraphs - Complete Hiring Pipeline\n")
print("="*80)

# Test Case 1: Successful candidate
print("\nTest Case 1: Processing Priya Sharma")
print("-" * 80)

pipeline_input = {
    "candidate_name": "Priya Sharma",
    "resume": SAMPLE_RESUME,
    "job_description": JOB_DESCRIPTION,
    "screening_result": "",
    "interview_scheduled": False,
    "interview_details": "",
    "final_status": ""
}

# Run the complete pipeline
result = hiring_pipeline.invoke(pipeline_input)

print("\n" + "="*80)
print("PIPELINE RESULT")
print("="*80)
print(result["final_status"])

## 🎯 Summary of Labs

We've built a complete progression of multi-agent systems:

### Lab 1: Basic Multi-Agent
- ✓ Multiple specialized agents working in sequence
- ✓ No external tools or memory
- ✓ Simple state passing between agents

### Lab 2: Multi-Agent with Tools
- ✓ Agents using external tools (HR database, email, calendar)
- ✓ Tool-calling pattern with LangChain
- ✓ Real-world HR operations

### Lab 3: Multi-Agent with Tools and Memory
- ✓ Persistent conversation memory
- ✓ Context retention across multiple turns
- ✓ Checkpointing with MemorySaver

### Lab 4: Human-in-the-Loop
- ✓ Approval workflows
- ✓ Human oversight and intervention
- ✓ Conditional workflow based on human decisions

### Lab 5: LangGraph Subgraphs
- ✓ Complex multi-agent architecture
- ✓ Nested subgraphs for team organization
- ✓ Parent-child graph communication
- ✓ Complete hiring pipeline orchestration

## 🚀 Next Steps

1. **Add Real Integrations**: Connect to actual HR systems (ATS, HRIS)
2. **Enhance Tools**: Add more sophisticated HR tools (background checks, assessments)
3. **Improve Memory**: Use persistent databases (PostgreSQL, MongoDB)
4. **Add Evaluation**: Implement metrics to measure agent performance
5. **Deploy**: Move to production with proper error handling and monitoring

## 📚 References

- [LangChain Multi-Agent Docs](https://docs.langchain.com/oss/python/langchain/multi-agent)
- [LangGraph Subgraphs](https://docs.langchain.com/oss/python/langgraph/use-subgraphs)
- [Human-in-the-Loop](https://docs.langchain.com/oss/python/langchain/human-in-the-loop)
- [LangChain Tools](https://docs.langchain.com/oss/python/langchain/tools)